# Prerequisites


In [2]:
# import libraries
import pandas as pd
import gurobipy as gp
import numpy as np
from sklearn.linear_model import LassoCV, Lasso

In [3]:
# read csv
train = pd.read_csv("training_data.csv")
test = pd.read_csv("test_data.csv")

# Gurobi Template for MIQP


In [ ]:
def miqp(k, batches):
    # batches is a 2D array comprised of all the batches we're not trying to solve the error for, should have len of 225 (9*25)
    M = 30
    timelimit = 3600  # in units of seconds
    # init model, betas, and z
    model = gp.Model()
    beta = model.addMVar(51, lb=-np.inf)
    z = model.addMVar(50, vtype="B")
    # obj, let i be the row within our batch, and j be each column in the row, loop through all 51 data points (j) in every row (i)
    model.setObjective(
        gp.quicksum(
            (
                beta[0]
                + gp.quicksum(
                    beta[j] * batches[i][j] for j in range(1, len(batches[i]))
                )
                - batches[i][0]
            )
            * (
                beta[0]
                + gp.quicksum(
                    beta[j] * batches[i][j] for j in range(1, len(batches[i]))
                )
                - batches[i][0]
            )
            for i in range(len(batches))
        )
    )
    # constraints
    model.addConstr(gp.quicksum(z[i] for i in range(50)) <= k)
    model.addConstrs(beta[i] <= z[i - 1] * M for i in range(1, 51))
    model.addConstrs(beta[i] >= z[i - 1] * -M for i in range(1, 51))
    model.write("model.lp")
    model.Params.OutputFlag = 0
    model.setParam("TimeLimit", timelimit)  # 1 hour per miqp
    model.optimize()
    # get and return betas for current batch so we can fit on batch we didn't include to get squared error
    return beta.x

# New Batch Function


In [ ]:
def get_new_batches(batches, n):
    # inputs all batches[i] where i != n into a tuple to add to a numpy array
    batches_to_add = tuple([batches[i] for i in range(len(batches)) if i != n])
    new_batch = np.concatenate(batches_to_add)
    return new_batch

# Get Squared Error Function


In [ ]:
def get_squared_error(batch, beta_list):
    # let batch be the batch that was not fitted to get betas
    # let beta_list be betas acquired from fitting all other batches
    squared_error = 0
    for i in batch:
        # (y - yhat)^2 | yhat = B0 + sum(all the betas * all the x's of batch) of that particular row
        yhat = beta_list[0] + sum(np.multiply(beta_list[1:], i[1:]))
        squared_error += np.square(i[0] - yhat)

    return squared_error

# Cross Validation


In [ ]:
# cross validation function
def cross_validation(batches):
    # let batches be a 3D array: the training data split into 10 batches with 25 rows in every batch and 51 data points (1 y, 50 x's) in every row
    k_list = []  # list to grab all k's and their squared errors
    for k in range(5, 55, 5):
        sum_squared_error = 0  # store sum of squared errors for a particular k
        # 10 fold cross validation
        for i in range(10):
            # get all batches excluding batches[i]
            new_batches = get_new_batches(batches, i)
            # get betas fit from iteration of cross validation
            beta_list = miqp(k, new_batches)
            # get squared error using betas fit from 9 other batches to batch not included in new_batches
            sum_squared_error += get_squared_error(batches[i], beta_list)
        # append k and squared array into k_list
        k_list.append((k, sum_squared_error))

    return k_list

In [ ]:
# shuffle rows and batch rows into clusters of 10 with length 25
training_rows = np.array(
    [train.iloc[i] for i in range(train.shape[0])]
)  # get all rows from training data into 1 array
np.random.shuffle(training_rows)  # shuffle
train_batches = np.array_split(
    training_rows, 10
)  # batch/cluster into 10 batches/clusters

In [ ]:
# get best value of k
k_list = cross_validation(train_batches)  # get all k's and their sum of squared errors
best_k = sorted(k_list, key=lambda x: x[1])[0][0]  # get best k

# display best_k and its mean squared error
print(
    f"the best k: {best_k}",
    f"cross validation results:",
    np.array([[x, y / train.shape[0]] for x, y in k_list]),
    sep="\n",
)

In [ ]:
# get best_k number of betas fit on the entire training set
training_set = np.array(
    [train.iloc[i] for i in range(train.shape[0])]
)  # get all rows from training data into 1 array
testing_set = np.array(
    [test.iloc[i] for i in range(test.shape[0])]
)  # get all rows from testing data into 1 array

training_betas = miqp(best_k, training_set)
direct_variable_squared_error = get_squared_error(testing_set, training_betas)
# mean squared error
direct_variable_squared_error / len(testing_set)

# Lasso Function


In [ ]:
# get prereq
x_train = [train.iloc[i][1:] for i in range(train.shape[0])]
y_train = [train.iloc[i][0] for i in range(train.shape[0])]
# get lambda/alpha with cross validation
lasso = LassoCV(cv=10).fit(x_train, y_train)
# show intercept & coefficients
lasso_model = Lasso(alpha=lasso.alpha_).fit(
    x_train, y_train
)  # 0.07638765995113507 = lambda/alpha
(lasso_model.intercept_, lasso_model.coef_)

In [ ]:
# get predictions
x_test = [test.iloc[i][1:] for i in range(test.shape[0])]
test_predictions = lasso_model.predict(x_test)

# get squared error of predictions
lasso_squared_error = 0
for i in range(test.shape[0]):
    lasso_squared_error += np.square(test.iloc[i][0] - test_predictions[i])
# mean squared error
lasso_squared_error / len(test_predictions)